In [1]:
import spacy

In [20]:
nlp = spacy.load("output/model-best")

c:\Users\paoma\miniconda3\lib\site-packages\spacy_transformers\layers\hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fi

In [70]:
text = "kontrak yg dr tangerang, jakarta dan alam sutera"

doc = nlp(text)

[(w.text, w.label_) for w in doc.ents]

[('kontrak', 'SUBJECT'),
 ('tangerang', 'VALUE'),
 ('jakarta', 'VALUE'),
 ('alam sutera', 'VALUE')]

Four labels for now: subject, attribute, logic and value

```sql
SELECT * FROM (subject) WHERE (attribute) (logic) '(value)'
```

In [71]:
subject = ""; attribute = ""; logic = ""; values = []

# transfer all entities into a single array
entities = [(w.text, w.label_) for w in doc.ents]

# get the subject (assume only one)
for w in entities:
    if w[1] == "SUBJECT":
        subject = w[0]
        break

# get the very first attribute
for w in entities:
    if w[1] == "ATTRIBUTE":
        old_att = w[0]
        break

# get the very first logic
for w in entities:
    if w[1] == "LOGIC":
        old_logic = w[0]
        break


# create an array that holds attribute, logic and value for a single SQL command
arguments = []

entities.append(("", "ATTRIBUTE")) # to trigger append for last entry

for w in entities:
    if w[1] == "ATTRIBUTE":
        if old_att == w[0]:
            attribute = w[0]

            if attribute[-3:] == "nya":
                attribute = attribute[:-3]

            old_att = attribute
        else:
            arguments.append([attribute, logic, values])
            attribute = ""; logic = ""; values = []
    if w[1] == "LOGIC":
        if old_logic == w[0] or w[0] == "sama":
            logic = w[0]

            if logic == "kurang" or "bawah" in logic:
                logic = "<"
                if logic == "sama":
                    logic += "="
            elif logic == "lebih" or "atas" in logic:
                logic = ">"
                if logic == "sama":
                    logic += "="
            elif "antara" in logic:
                logic = "AND" 

            old_logic = logic
        else:
            arguments.append([attribute, logic, values])
            attribute = ""; logic = ""; values = []
    if w[1] == "VALUE":
        values.append(w[0])

# clean up empty values
for arg in arguments:
    if arg[0] == "":
        arg[0] = "location"
    if arg[1] == "":
        arg[1] = "in"


# create sql statement
sql = f"SELECT * FROM {subject} WHERE"

for arg in arguments:
    # for between arguments
    if arg[1] == "AND":
        sql += f" {arg[0]} BETWEEN '{arg[2][0]}' {arg[1]} '{arg[2][1]}'"
    # for others
    else:
        vals = "("
        for val in arg[2]:
            if val.isnumeric():
                vals += val + ","
            else:
                vals += f"'{val}',"
        vals = vals[:-1] + ")"

        sql += f" {arg[0]} {arg[1]} {vals}"

sql += ";" # to define closure of an sql statement

print(sql)

SELECT * FROM kontrak WHERE location in ('tangerang','jakarta','alam sutera');
